# Conclusion

Nous avons utilisé la technique de bagging ici sans la validation croisée.

**Amélioration de la Performance**
- Accuracy: L'accuracy de votre modèle est passée de 0.8127 à 0.8290, ce qui représente une amélioration plus de 1%. Cela indique que le modèle après le bagging est capable de prédire plus précisément les étiquettes de classe.

- Macro F1 Score: Le macro F1 score a légèrement diminué, passant de 0.7888 à 0.7875. Bien que le macro F1 score ait légèrement diminué, ce changement est relativement mineur.

En créant plusieurs sous-modèles entraînés sur des sous-ensembles des données, ce qui augmente la diversité parmi les prédictions. En combinant les prédictions de ces sous-modèles, le modèle final est capable de mieux généraliser sur l'ensemble des données, réduisant ainsi les erreurs dues à la variance. 

- Micro F1 Score: Ce score reflète également une amélioration, de 0.8127 à 0.8290, similaire à l'accuracy, montrant une meilleure capacité du modèle à généraliser sur les données.

**Amélioration de l'Équité**
- TPR_GAP (True Positive Rate Gap): Il y a eu une légère réduction du TPR_GAP, de 0.1738 à 0.1638, ce qui indique une amélioration de l'équité du modèle en termes de taux de vrais positifs entre les groupes. La diminution est d'environ 1.0%, améliorant ainsi l'équité du modèle.

- FPR_GAP (False Positive Rate Gap): La différence entre les taux de faux positifs a également légèrement diminué, de 0.00395 à 0.00361. Bien que le changement soit petit, toute amélioration dans la réduction des disparités est positive.

Ceci est dû au fait que différentes instances de sous-modèles sont exposées à différentes parties des données, ce qui peut aider à réduire les biais spécifiques à un sous-ensemble de données. Ainsi, certains modèles peuvent apprendre à mieux classer les instances où d'autres modèles avaient des difficultés.

- PPR_GAP (Predicted Positive Rate Gap): Ce paramètre a vu une augmentation, de 0.0043 à 0.0200. Cette augmentation suggère une répartition moins équitable des prédictions positives entre les différents groupes. Ce qui pourrait indiquer une zone d'amélioration pour le modèle bagging.

L'augmentation du PPR_GAP pourrait être due à une répartition inégale des prédictions positives parmi les différents sous-modèles, surtout si certains groupes ou caractéristiques sont mal représentés dans les ensembles de données. Cela souligne l'importance d'une analyse approfondie de l'échantillonnage des données pour le bagging afin de garantir une représentation juste.

Score Final
Le score final, calculé à partir du macro F1 score et de l'inverse du TPR_GAP, a augmenté de 0.8075 à 0.8118. Cela indique une amélioration globale en termes de performance et d'équité, malgré la légère baisse du macro F1 score et l'augmentation du PPR_GAP.

Ce modèle de bagging pourrait être amélioré tout comme la possibilité d'entrainer d'autres familles de méthodes. Mais le temps n'est pas suffisant. Néanmoins, on a réussit à entrainer un modèle qui présentant un score de 72 sur le test jusqu'à 78,

In [29]:
import pandas as pd
from modelization import *
import pandas as pd
from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from evaluator import *

In [4]:
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)

In [5]:
# Features
X_train = dat['X_train']

# Données à prédire
Y_train = dat['Y']

# Attributs sensibles
S_train = dat['S_train']

In [6]:
X_test = dat['X_test'] 
S_test = dat['S_test']

In [7]:
import pandas as pd
from sklearn.utils import resample

# Supposons que X_train, Y_train, et S_train sont déjà définis comme dans votre question
data = pd.concat([S_train, X_train, Y_train],axis=1)

balanced_data_list = []
for nbclass in range(28):  # Assumant 28 classes professionnelles distinctes
    # Sélection des données pour une classe professionnelle spécifique
    data_class = data[data['profession_class'] == nbclass]
    
    # Séparation basée sur la classe de genre
    data_s0 = data_class[data_class['gender_class'] == 0]
    data_s1 = data_class[data_class['gender_class'] == 1]
    
    if len(data_s0) >= len(data_s1):
        # Rééchantillonnage pour équilibrer les classes de genre
        data_s1_resampled = resample(data_s1, replace=True, n_samples=len(data_s0), random_state=42)
        balanced_data = pd.concat([data_s0, data_s1_resampled])
    else:
        data_s0_resampled = resample(data_s0, replace=True, n_samples=len(data_s1), random_state=42)
        balanced_data = pd.concat([data_s1, data_s0_resampled])
        
    balanced_data_list.append(balanced_data)

# Combiner toutes les données rééquilibrées en un seul DataFrame
balanced_data_combined = pd.concat(balanced_data_list)

In [6]:
balanced_data_combined

,gender_class,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,profession_class
15102,0,-0.106644,0.206202,-0.487275,-0.565128,0.002457,0.521392,0.222879,0.332393,-0.158473,...,-0.025967,-0.188476,-0.541974,0.066505,-0.000817,-0.237024,-0.169442,0.153728,0.160571,0
9853,0,-0.539986,0.367027,-1.042473,-0.315929,0.223228,-0.020717,-0.085954,0.298949,-0.063420,...,-0.231364,0.026662,-0.298197,0.536527,-0.356383,0.343746,0.356660,0.265317,0.140032,0
38521,0,-0.667977,0.534055,-0.372200,-0.912800,-0.162027,0.316608,0.409040,0.698239,-0.228332,...,-0.174538,-0.090482,-0.540254,0.252228,0.325463,-0.212083,0.143992,-0.016629,0.306894,0
22478,0,-0.646435,0.244130,-0.838173,-0.224850,-0.186924,0.327114,-0.150429,0.745634,-0.238859,...,-0.076886,0.060196,0.024156,0.312725,0.224630,-0.279840,0.124797,0.418145,0.388695,0
28294,0,-0.269952,0.055027,-0.498366,-0.789691,0.517833,0.072657,0.168787,0.356092,-0.075094,...,-0.280233,0.377222,-0.733948,0.610944,-0.117537,-0.160587,0.328925,0.379089,0.035509,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,0,-0.150664,-0.275969,0.328843,-0.289479,-0.359328,-0.589007,0.828748,0.445308,-0.073924,...,-0.451324,-0.064055,-0.831989,0.462128,0.823785,-0.522116,-0.273711,-0.010537,0.610961,27
4712,0,-0.696221,0.049864,-0.205544,-0.244224,-0.378419,-0.132278,0.664804,0.635028,0.081595,...,-0.171151,0.397429,-0.452958,0.146219,0.330908,-0.029582,-0.276181,0.445554,-0.012045,27
22208,0,-0.071043,0.044882,-0.311584,-0.620255,-0.161639,0.015657,0.541004,0.306950,0.011576,...,-0.627619,-0.025529,-0.389166,-0.107837,0.682593,-0.051020,0.044216,0.269877,0.008501,27
22664,0,-0.269814,-0.132485,0.134214,-0.446334,-0.440597,-0.351853,0.659138,0.553297,-0.405849,...,0.016632,0.678242,-0.739566,0.369372,0.412901,-0.314313,-0.157456,0.622148,-0.006625,27


In [9]:
# Assuming balanced_data_combined is a pandas DataFrame
# Supprimer la première colonne
X_balanced = balanced_data_combined.iloc[:, 1:]

# Supprimer la dernière colonne
X_balanced = X_balanced.iloc[:, :-1]

S_balanced = balanced_data_combined.iloc[:, 0]

Y_balanced = balanced_data_combined.iloc[:, -1]  # Select all rows for the second-last column

In [10]:
X_balanced

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
15102,-0.106644,0.206202,-0.487275,-0.565128,0.002457,0.521392,0.222879,0.332393,-0.158473,-0.425709,...,0.262542,-0.025967,-0.188476,-0.541974,0.066505,-0.000817,-0.237024,-0.169442,0.153728,0.160571
9853,-0.539986,0.367027,-1.042473,-0.315929,0.223228,-0.020717,-0.085954,0.298949,-0.063420,-0.127194,...,0.263436,-0.231364,0.026662,-0.298197,0.536527,-0.356383,0.343746,0.356660,0.265317,0.140032
38521,-0.667977,0.534055,-0.372200,-0.912800,-0.162027,0.316608,0.409040,0.698239,-0.228332,0.177368,...,0.074018,-0.174538,-0.090482,-0.540254,0.252228,0.325463,-0.212083,0.143992,-0.016629,0.306894
22478,-0.646435,0.244130,-0.838173,-0.224850,-0.186924,0.327114,-0.150429,0.745634,-0.238859,0.379400,...,0.040561,-0.076886,0.060196,0.024156,0.312725,0.224630,-0.279840,0.124797,0.418145,0.388695
28294,-0.269952,0.055027,-0.498366,-0.789691,0.517833,0.072657,0.168787,0.356092,-0.075094,-0.683792,...,0.199538,-0.280233,0.377222,-0.733948,0.610944,-0.117537,-0.160587,0.328925,0.379089,0.035509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,-0.150664,-0.275969,0.328843,-0.289479,-0.359328,-0.589007,0.828748,0.445308,-0.073924,-0.626374,...,0.219889,-0.451324,-0.064055,-0.831989,0.462128,0.823785,-0.522116,-0.273711,-0.010537,0.610961
4712,-0.696221,0.049864,-0.205544,-0.244224,-0.378419,-0.132278,0.664804,0.635028,0.081595,-0.573454,...,-0.403174,-0.171151,0.397429,-0.452958,0.146219,0.330908,-0.029582,-0.276181,0.445554,-0.012045
22208,-0.071043,0.044882,-0.311584,-0.620255,-0.161639,0.015657,0.541004,0.306950,0.011576,-0.167356,...,0.098436,-0.627619,-0.025529,-0.389166,-0.107837,0.682593,-0.051020,0.044216,0.269877,0.008501
22664,-0.269814,-0.132485,0.134214,-0.446334,-0.440597,-0.351853,0.659138,0.553297,-0.405849,-0.363420,...,-0.137180,0.016632,0.678242,-0.739566,0.369372,0.412901,-0.314313,-0.157456,0.622148,-0.006625


In [11]:
S_balanced

15102    0
9853     0
38521    0
22478    0
28294    0
        ..
4076     0
4712     0
22208    0
22664    0
19012    0
Name: gender_class, Length: 34704, dtype: int64

In [12]:
Y_balanced

15102     0
9853      0
38521     0
22478     0
28294     0
         ..
4076     27
4712     27
22208    27
22664    27
19012    27
Name: profession_class, Length: 34704, dtype: int64

# bagging

In [ ]:
regression_classique = result_3[0].predict(X_test.values)
results=pd.DataFrame(regression_classique, columns= ['score'])

results.to_csv("Data_Challenge_MDI_33.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')
#0.

In [36]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
logistic_regression = LogisticRegression(random_state=42, solver='lbfgs', C=0.09, multi_class='multinomial', max_iter=5000)

#BaggingClassifier et Logistic Regression
bagging_logistic = BaggingClassifier(logistic_regression, n_estimators=20)

# BaggingClassifier
bagging_logistic.fit(X_balanced, Y_balanced)
_, X_val, _, y_val, _, S_val = train_test_split(X_train, Y_train, S_train ,test_size=0.2, random_state=42)

# prédiction 
y_pred = bagging_logistic.predict(X_val)

In [38]:
bagging_logistic

BaggingClassifier(estimator=LogisticRegression(C=0.09, max_iter=5000,
                                               multi_class='multinomial',
                                               random_state=42),
                  n_estimators=20)

In [39]:
X_true = dat['X_test']
S_true = dat['S_test'] 
data_true = pd.concat([S_true, X_true], axis=1)
# Classify the provided test data with you classifier
y_pref_true = bagging_logistic.predict(X_true)
results=pd.DataFrame(y_pref_true, columns= ['score'])

results.to_csv("Data_Challenge_MDI_31.csv", header = None, index = None)

In [41]:
# Évaluation des performances et de l'équité du modèle
eval_scores, confusion_matrices = gap_eval_scores(y_pred, y_val, S_val, metrics=["TPR", "FPR", "PPR"])

# Affichage des scores de performance
print("Scores de performance et d'équité:")
print(eval_scores)

Scores de performance et d'équité:
{'accuracy': 0.829009009009009, 'macro_fscore': 0.7874693128672671, 'micro_fscore': 0.829009009009009, 'TPR_GAP': 0.16378340728519633, 'FPR_GAP': 0.0036056692045361356, 'PPR_GAP': 0.019969158367083142}


In [42]:
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
print(final_score)

0.8118429527910354
